# Model Testing

Welcome to the 3rd part of Utilizing Artificial Intelligence for Algorithmic Trading.
For this part we will be testing the models built in step 2. This step will also introduce the sentiment models that do not require testing.
The result of this script will be a .csv that will be used in step 4 where we build the meta models.

This code has been built for a large scale research project for Texas State University.
The research for the project has been conducted by James Pavlicek, Jack Burt, and Andrew Hocher.
If you have any questions or inquiries feel free to reach out to me at https://www.jamespavlicek.com/ 
This code is free use and anyone can use it.

# Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

import re
import numpy as np  
import pandas as pd
import nltk

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer

import re, string, unicodedata 
import contractions
from bs4 import BeautifulSoup

import numpy as np 
import pandas as pd 
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize 
from nltk.stem.wordnet import WordNetLemmatizer 
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report 
from sklearn.model_selection import cross_val_score

from joblib import dump
from joblib import load

# Read Dataset

In [2]:
#This is the dataset from the train/test split in step 2. Remember to set your own path.
data = pd.read_csv("/Users/jamespavlicek/Desktop/QMST/4320/Project Final/test_data.csv")

df = data.copy()
df_2 = df.copy()

# Data Summary

In [3]:
df.shape

(6082, 9)

In [4]:
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

There are 6082 rows and 9 columns.


In [5]:
df.head()

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Largest Text,Ticker,Percent Change (1 Week),Return Type
0,20211214_10-K_edgar_data_827876_0001663577-21-...,"cleanspark, inc.",20211214,827876,services-computer integrated systems design [7...,item 7. management s discussion and analysis o...,CLSK,-0.106524,Negative
1,20220225_10-K_edgar_data_1046025_0001046025-22...,heritage financial corp /wa/,20220225,1046025,"savings institutions, not federally chartered ...",item 7. management's discussion and analysis o...,HFWA,0.008755,Positive
2,20210202_10-K_edgar_data_822416_0000822416-21-...,pultegroup inc/mi/,20210202,822416,operative builders [1531],item 7. management s discussion and analysis o...,PHM,0.071569,Positive
3,20190222_10-K_edgar_data_1099219_0001099219-19...,metlife inc,20190222,1099219,life insurance [6311],item 7. management s discussion and analysis o...,MET,0.010058,Positive
4,20230214_10-K_edgar_data_1109357_0001109357-23...,exelon corp,20230214,1109357,electric & other services combined [4931],item 7. management's discussion and analysis o...,EXC,0.024268,Positive


In [6]:
df.tail()

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Largest Text,Ticker,Percent Change (1 Week),Return Type
6077,20200218_10-K_edgar_data_1360604_0001360604-20...,"healthcare trust of america, inc.",20200218,1360604,real estate investment trusts [6798],item 7. management s discussion and analysis o...,HR,0.013804,Positive
6078,20230228_10-K_edgar_data_1411579_0001411579-23...,"amc entertainment holdings, inc.",20230228,1411579,services-motion picture theaters [7830],item 7. management s discussion and analysis o...,AMC,-0.124650,Negative
6079,20170228_10-K_edgar_data_1163302_0001163302-17...,united states steel corp,20170228,1163302,"steel works, blast furnaces rolling mills (co...",item 7. management's discussion and analysis o...,X,-0.032025,Negative
6080,20090331_10-K_edgar_data_43196_0000950144-09-0...,gray television inc,20090331,43196,television broadcasting stations [4833],item 7. management s discussion and analysis o...,GTN,0.406250,Positive
6081,20200316_10-K_edgar_data_1468748_0001564590-20...,kodiak sciences inc.,20200316,1468748,biological products (no diagnostic substances)...,item 7. management s discussion and analys is ...,KOD,0.116291,Positive


In [7]:
np.random.seed(2)
df.sample(10)

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Largest Text,Ticker,Percent Change (1 Week),Return Type
2482,20210310_10-K_edgar_data_1747079_0001747079-21...,bally's corp,20210310,1747079,hotels & motels [7011],item 7. management s discussion and analysis o...,BALY,0.003616,Positive
4189,20140415_10-K_edgar_data_1108046_0001477932-14...,alumifuel power corp,20140415,1108046,industrial organic chemicals [2860],item 7. management s discussion and analysis o...,AFPW,-0.125000,Negative
2806,20170331_10-K_edgar_data_829325_0001262463-17-...,"spyr, inc.",20170331,829325,retail-eating & drinking places [5810],item 7. management s discussion and analysis o...,SPYR,-0.001857,Negative
625,20140319_10-K_edgar_data_1364479_0001364479-14...,hertz global holdings inc,20140319,1364479,services-auto rental & leasing (no drivers) [7...,item 7. management's discussion and analysis o...,HRI,-0.032139,Negative
5261,20170228_10-K_edgar_data_1093557_0001093557-17...,dexcom inc,20170228,1093557,surgical & medical instruments & apparatus [3841],item 7. management s discussion and analysis o...,DXCM,0.002559,Positive
2187,20140227_10-K_edgar_data_60519_0001504337-14-0...,louisiana-pacific corp,20140227,60519,lumber & wood products (no furniture) [2400],item 7. management s discussion and analysis o...,LPX,-0.019282,Negative
1558,20230314_10-K_edgar_data_771266_0001493152-23-...,kopin corp,20230314,771266,semiconductors & related devices [3674],item 7. management s discussion and analysis o...,KOPN,-0.049505,Negative
889,20211215_10-K_edgar_data_1410738_0001493152-21...,voip-pal.com inc,20211215,1410738,telephone & telegraph apparatus [3661],item 7. management s discussion and analysis o...,VPLM,-0.076923,Negative
463,20220517_10-K_edgar_data_946581_0001628280-22-...,take two interactive software inc,20220517,946581,services-prepackaged software [7372],item 7. management's discussion and analysis o...,TTWO,0.004387,Positive
5970,20020322_10-K_edgar_data_833640_0001012870-02-...,power integrations inc,20020322,833640,semiconductors & related devices [3674],item 7. management's discussion and analysis o...,POWI,0.014917,Positive


In [8]:
df[data.duplicated()].count()

Filename                              0
Company Name                          0
Filed As Of Date                      0
Central Index Key                     0
Standard Industrial Classification    0
Largest Text                          0
Ticker                                0
Percent Change (1 Week)               0
Return Type                           0
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6082 entries, 0 to 6081
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Filename                            6082 non-null   object 
 1   Company Name                        6082 non-null   object 
 2   Filed As Of Date                    6082 non-null   int64  
 3   Central Index Key                   6082 non-null   int64  
 4   Standard Industrial Classification  6082 non-null   object 
 5   Largest Text                        6082 non-null   object 
 6   Ticker                              6082 non-null   object 
 7   Percent Change (1 Week)             6082 non-null   float64
 8   Return Type                         6082 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 427.8+ KB


In [10]:
df.isnull().sum()

Filename                              0
Company Name                          0
Filed As Of Date                      0
Central Index Key                     0
Standard Industrial Classification    0
Largest Text                          0
Ticker                                0
Percent Change (1 Week)               0
Return Type                           0
dtype: int64

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Filed As Of Date,6082.0,2.016275e+07,60619.844668,2.000022e+07,2.013021e+07,2.018030e+07,2.021032e+07,2.023123e+07
Central Index Key,6082.0,1.002136e+06,522685.163134,1.750000e+03,7.977210e+05,1.046568e+06,1.409056e+06,1.907982e+06
Percent Change (1 Week),6082.0,8.449757e-05,0.122309,-6.949153e-01,-3.241883e-02,-5.594605e-04,2.729096e-02,4.869565e+00


# Preprocessing

## Tag Removal

In [12]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

df["Largest Text"] = df["Largest Text"].apply(lambda x: strip_html(x))

df.head()

mean_length = df['Largest Text'].apply(len).mean()
print(f"The mean length of the 'Largest Text' entries is: {mean_length}")

The mean length of the 'Largest Text' entries is: 105697.17264057876


## Replace Contractions 

In [13]:
def replace_contractions(text):
    return contractions.fix(text)

df["Largest Text"] = df["Largest Text"].apply(lambda x: replace_contractions(x))

df.head()

df_3 = df.copy()

## Remove Numbers/Special Characters 

In [15]:
def remove_special_characters(text, remove_digits=True):
    special = r"[^a-zA-Z0-9\s]" if not remove_digits else r"[^a-zA-z\s]"
    text = re.sub(special, "", text)
    return text

df["Largest Text"] = df["Largest Text"].apply(lambda x: remove_special_characters(x))

df.head()

mean_length = df['Largest Text'].apply(len).mean()
print(f"The mean length of the 'Largest Text' entries is: {mean_length}")


The mean length of the 'Largest Text' entries is: 99054.97435054259


## Conversion to Lowercase 

In [16]:
def to_lowercase(text):
    lower = text.lower()
    return lower

df["Largest Text"] = df["Largest Text"].apply(lambda x: to_lowercase(x))

df.head()

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Largest Text,Ticker,Percent Change (1 Week),Return Type
0,20211214_10-K_edgar_data_827876_0001663577-21-...,"cleanspark, inc.",20211214,827876,services-computer integrated systems design [7...,item management s discussion and analysis of ...,CLSK,-0.106524,Negative
1,20220225_10-K_edgar_data_1046025_0001046025-22...,heritage financial corp /wa/,20220225,1046025,"savings institutions, not federally chartered ...",item managements discussion and analysis of f...,HFWA,0.008755,Positive
2,20210202_10-K_edgar_data_822416_0000822416-21-...,pultegroup inc/mi/,20210202,822416,operative builders [1531],item management s discussion and analysis of ...,PHM,0.071569,Positive
3,20190222_10-K_edgar_data_1099219_0001099219-19...,metlife inc,20190222,1099219,life insurance [6311],item management s discussion and analysis of ...,MET,0.010058,Positive
4,20230214_10-K_edgar_data_1109357_0001109357-23...,exelon corp,20230214,1109357,electric & other services combined [4931],item managements discussion and analysis of f...,EXC,0.024268,Positive


## Tokenization

In [17]:
df['Largest Text'] = df['Largest Text'].astype(str)
for index, row in df.iterrows():
    df.at[index, "Largest Text"] = nltk.word_tokenize(row["Largest Text"])
df.head()

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Largest Text,Ticker,Percent Change (1 Week),Return Type
0,20211214_10-K_edgar_data_827876_0001663577-21-...,"cleanspark, inc.",20211214,827876,services-computer integrated systems design [7...,"[item, management, s, discussion, and, analysi...",CLSK,-0.106524,Negative
1,20220225_10-K_edgar_data_1046025_0001046025-22...,heritage financial corp /wa/,20220225,1046025,"savings institutions, not federally chartered ...","[item, managements, discussion, and, analysis,...",HFWA,0.008755,Positive
2,20210202_10-K_edgar_data_822416_0000822416-21-...,pultegroup inc/mi/,20210202,822416,operative builders [1531],"[item, management, s, discussion, and, analysi...",PHM,0.071569,Positive
3,20190222_10-K_edgar_data_1099219_0001099219-19...,metlife inc,20190222,1099219,life insurance [6311],"[item, management, s, discussion, and, analysi...",MET,0.010058,Positive
4,20230214_10-K_edgar_data_1109357_0001109357-23...,exelon corp,20230214,1109357,electric & other services combined [4931],"[item, managements, discussion, and, analysis,...",EXC,0.024268,Positive


In [18]:
mean_length = df['Largest Text'].apply(len).mean()
print(f"The mean length of the 'Largest Text' entries is: {mean_length}")

The mean length of the 'Largest Text' entries is: 14714.792009207498


## Remove Stopwords

In [19]:
stopwords = stopwords.words("english")

customlist = [
    "not",
    "could",
    "did",
    "does",
    "had",
    "has",
    "have",
    "is",
    "ma",
    "might",
    "must",
    "need",
    "shall",
    "should",
    "was",
    "were",
    "will",
    "would",
]

stopwords = list(set(stopwords) - set(customlist))

In [20]:
def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopwords:
            new_words.append(word)
    return new_words

df["Largest Text"] = df["Largest Text"].apply(lambda x: remove_stopwords(x))

df.head()

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Largest Text,Ticker,Percent Change (1 Week),Return Type
0,20211214_10-K_edgar_data_827876_0001663577-21-...,"cleanspark, inc.",20211214,827876,services-computer integrated systems design [7...,"[item, management, discussion, analysis, finan...",CLSK,-0.106524,Negative
1,20220225_10-K_edgar_data_1046025_0001046025-22...,heritage financial corp /wa/,20220225,1046025,"savings institutions, not federally chartered ...","[item, managements, discussion, analysis, fina...",HFWA,0.008755,Positive
2,20210202_10-K_edgar_data_822416_0000822416-21-...,pultegroup inc/mi/,20210202,822416,operative builders [1531],"[item, management, discussion, analysis, finan...",PHM,0.071569,Positive
3,20190222_10-K_edgar_data_1099219_0001099219-19...,metlife inc,20190222,1099219,life insurance [6311],"[item, management, discussion, analysis, finan...",MET,0.010058,Positive
4,20230214_10-K_edgar_data_1109357_0001109357-23...,exelon corp,20230214,1109357,electric & other services combined [4931],"[item, managements, discussion, analysis, fina...",EXC,0.024268,Positive


## Lemmatize

In [21]:
lemmatizer = WordNetLemmatizer()

def lemmatize_list(words):
    new_words = []
    for word in words:
        new_words.append(lemmatizer.lemmatize(word, pos="v"))
    return new_words

df["Largest Text"] = df["Largest Text"].apply(lambda x: lemmatize_list(x))

df.head()

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Largest Text,Ticker,Percent Change (1 Week),Return Type
0,20211214_10-K_edgar_data_827876_0001663577-21-...,"cleanspark, inc.",20211214,827876,services-computer integrated systems design [7...,"[item, management, discussion, analysis, finan...",CLSK,-0.106524,Negative
1,20220225_10-K_edgar_data_1046025_0001046025-22...,heritage financial corp /wa/,20220225,1046025,"savings institutions, not federally chartered ...","[item, managements, discussion, analysis, fina...",HFWA,0.008755,Positive
2,20210202_10-K_edgar_data_822416_0000822416-21-...,pultegroup inc/mi/,20210202,822416,operative builders [1531],"[item, management, discussion, analysis, finan...",PHM,0.071569,Positive
3,20190222_10-K_edgar_data_1099219_0001099219-19...,metlife inc,20190222,1099219,life insurance [6311],"[item, management, discussion, analysis, finan...",MET,0.010058,Positive
4,20230214_10-K_edgar_data_1109357_0001109357-23...,exelon corp,20230214,1109357,electric & other services combined [4931],"[item, managements, discussion, analysis, fina...",EXC,0.024268,Positive


## Word List to Text String 

In [22]:
def join_words(words):
    return " ".join(words)

df["Largest Text"] = df["Largest Text"].apply(lambda x: join_words(x))

df.head()

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Largest Text,Ticker,Percent Change (1 Week),Return Type
0,20211214_10-K_edgar_data_827876_0001663577-21-...,"cleanspark, inc.",20211214,827876,services-computer integrated systems design [7...,item management discussion analysis financial ...,CLSK,-0.106524,Negative
1,20220225_10-K_edgar_data_1046025_0001046025-22...,heritage financial corp /wa/,20220225,1046025,"savings institutions, not federally chartered ...",item managements discussion analysis financial...,HFWA,0.008755,Positive
2,20210202_10-K_edgar_data_822416_0000822416-21-...,pultegroup inc/mi/,20210202,822416,operative builders [1531],item management discussion analysis financial ...,PHM,0.071569,Positive
3,20190222_10-K_edgar_data_1099219_0001099219-19...,metlife inc,20190222,1099219,life insurance [6311],item management discussion analysis financial ...,MET,0.010058,Positive
4,20230214_10-K_edgar_data_1109357_0001109357-23...,exelon corp,20230214,1109357,electric & other services combined [4931],item managements discussion analysis financial...,EXC,0.024268,Positive


In [23]:
df.tail()

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Largest Text,Ticker,Percent Change (1 Week),Return Type
6077,20200218_10-K_edgar_data_1360604_0001360604-20...,"healthcare trust of america, inc.",20200218,1360604,real estate investment trusts [6798],item management discussion analysis financial ...,HR,0.013804,Positive
6078,20230228_10-K_edgar_data_1411579_0001411579-23...,"amc entertainment holdings, inc.",20230228,1411579,services-motion picture theaters [7830],item management discussion analysis financial ...,AMC,-0.124650,Negative
6079,20170228_10-K_edgar_data_1163302_0001163302-17...,united states steel corp,20170228,1163302,"steel works, blast furnaces rolling mills (co...",item managements discussion analysis financial...,X,-0.032025,Negative
6080,20090331_10-K_edgar_data_43196_0000950144-09-0...,gray television inc,20090331,43196,television broadcasting stations [4833],item management discussion analysis financial ...,GTN,0.406250,Positive
6081,20200316_10-K_edgar_data_1468748_0001564590-20...,kodiak sciences inc.,20200316,1468748,biological products (no diagnostic substances)...,item management discussion analys be financial...,KOD,0.116291,Positive


In [24]:
np.random.seed(2)
df.sample(10)

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Largest Text,Ticker,Percent Change (1 Week),Return Type
2482,20210310_10-K_edgar_data_1747079_0001747079-21...,bally's corp,20210310,1747079,hotels & motels [7011],item management discussion analysis financial ...,BALY,0.003616,Positive
4189,20140415_10-K_edgar_data_1108046_0001477932-14...,alumifuel power corp,20140415,1108046,industrial organic chemicals [2860],item management discussion analysis financial ...,AFPW,-0.125000,Negative
2806,20170331_10-K_edgar_data_829325_0001262463-17-...,"spyr, inc.",20170331,829325,retail-eating & drinking places [5810],item management discussion analysis financial ...,SPYR,-0.001857,Negative
625,20140319_10-K_edgar_data_1364479_0001364479-14...,hertz global holdings inc,20140319,1364479,services-auto rental & leasing (no drivers) [7...,item managements discussion analysis financial...,HRI,-0.032139,Negative
5261,20170228_10-K_edgar_data_1093557_0001093557-17...,dexcom inc,20170228,1093557,surgical & medical instruments & apparatus [3841],item management discussion analysis financial ...,DXCM,0.002559,Positive
2187,20140227_10-K_edgar_data_60519_0001504337-14-0...,louisiana-pacific corp,20140227,60519,lumber & wood products (no furniture) [2400],item management discussion analysis financial ...,LPX,-0.019282,Negative
1558,20230314_10-K_edgar_data_771266_0001493152-23-...,kopin corp,20230314,771266,semiconductors & related devices [3674],item management discussion analysis financial ...,KOPN,-0.049505,Negative
889,20211215_10-K_edgar_data_1410738_0001493152-21...,voip-pal.com inc,20211215,1410738,telephone & telegraph apparatus [3661],item management discussion analysis financial ...,VPLM,-0.076923,Negative
463,20220517_10-K_edgar_data_946581_0001628280-22-...,take two interactive software inc,20220517,946581,services-prepackaged software [7372],item managements discussion analysis financial...,TTWO,0.004387,Positive
5970,20020322_10-K_edgar_data_833640_0001012870-02-...,power integrations inc,20020322,833640,semiconductors & related devices [3674],item managements discussion analysis financial...,POWI,0.014917,Positive


## Encoded "Return Type" for different classification algorithms.
0 = negative
1 = positive 

In [25]:
df["Return Type"] = df["Return Type"].astype("category")
df["Return Type"] = df["Return Type"].cat.codes

df.head()

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Largest Text,Ticker,Percent Change (1 Week),Return Type
0,20211214_10-K_edgar_data_827876_0001663577-21-...,"cleanspark, inc.",20211214,827876,services-computer integrated systems design [7...,item management discussion analysis financial ...,CLSK,-0.106524,0
1,20220225_10-K_edgar_data_1046025_0001046025-22...,heritage financial corp /wa/,20220225,1046025,"savings institutions, not federally chartered ...",item managements discussion analysis financial...,HFWA,0.008755,1
2,20210202_10-K_edgar_data_822416_0000822416-21-...,pultegroup inc/mi/,20210202,822416,operative builders [1531],item management discussion analysis financial ...,PHM,0.071569,1
3,20190222_10-K_edgar_data_1099219_0001099219-19...,metlife inc,20190222,1099219,life insurance [6311],item management discussion analysis financial ...,MET,0.010058,1
4,20230214_10-K_edgar_data_1109357_0001109357-23...,exelon corp,20230214,1109357,electric & other services combined [4931],item managements discussion analysis financial...,EXC,0.024268,1


## Add New Columns to Dataframe to hold results

In [26]:
df['Random_Forest_CV'] = np.nan
df['Random_Forest_TF_IDF'] = np.nan
df['XGBoost_CV'] = np.nan
df['XGBoost_CV_TF_IDF'] = np.nan 
df['Naive_Bayes_CV'] = np.nan
df['Naive_Bayes_TF_IDF'] = np.nan
df['Positive_Word_Count_Loughran_McDonald'] = np.nan
df['Negative_Word_Count_Loughran_McDonald'] = np.nan
df['Total Word Count_Loughran_McDonald'] = np.nan
df["Word_Pos_minus_Neg_Score_Loughran_McDonald"] = np.nan 
df['Word_Sentiment_Type_Loughran_McDonald'] = np.nan
df['TextBlob_Sentiment_Score'] = np.nan  
df['TextBlob_Sentiment_Type'] = np.nan 
df['FinBERT_Positive_Sentence_Count'] = np.nan
df['FinBERT_Neutral_Sentence_Count'] = np.nan
df['FinBERT_Negative_Sentence_Count'] = np.nan
df["FinBERT_Pos_minus_Neg_Score"] = np.nan
df['FinBERT_Sentiment_Type'] = np.nan
df['Chat_GPT_Sentiment'] = np.nan
df['Chat_GPT_Sentiment_binary'] = np.nan

df.head()

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Largest Text,Ticker,Percent Change (1 Week),Return Type,Random_Forest_CV,...,Word_Sentiment_Type_Loughran_McDonald,TextBlob_Sentiment_Score,TextBlob_Sentiment_Type,FinBERT_Positive_Sentence_Count,FinBERT_Neutral_Sentence_Count,FinBERT_Negative_Sentence_Count,FinBERT_Pos_minus_Neg_Score,FinBERT_Sentiment_Type,Chat_GPT_Sentiment,Chat_GPT_Sentiment_binary
0,20211214_10-K_edgar_data_827876_0001663577-21-...,"cleanspark, inc.",20211214,827876,services-computer integrated systems design [7...,item management discussion analysis financial ...,CLSK,-0.106524,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20220225_10-K_edgar_data_1046025_0001046025-22...,heritage financial corp /wa/,20220225,1046025,"savings institutions, not federally chartered ...",item managements discussion analysis financial...,HFWA,0.008755,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210202_10-K_edgar_data_822416_0000822416-21-...,pultegroup inc/mi/,20210202,822416,operative builders [1531],item management discussion analysis financial ...,PHM,0.071569,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20190222_10-K_edgar_data_1099219_0001099219-19...,metlife inc,20190222,1099219,life insurance [6311],item management discussion analysis financial ...,MET,0.010058,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20230214_10-K_edgar_data_1109357_0001109357-23...,exelon corp,20230214,1109357,electric & other services combined [4931],item managements discussion analysis financial...,EXC,0.024268,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Vectorization

## CountVectorizer

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vec = CountVectorizer(max_features=4000)
data_features1 = bow_vec.fit_transform(df["Largest Text"])
data_features1 = data_features1.toarray() 

In [29]:
data_features1.shape

(6082, 4000)

In [30]:
X1 = data_features1
y1 = df["Return Type"]

## TF-IDF

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=4000)
data_features2 = vectorizer.fit_transform(df["Largest Text"])
data_features2 = data_features2.toarray()

In [32]:
data_features2.shape

(6082, 4000)

In [33]:
X2 = data_features2
y2 = df["Return Type"]

### 1 = CountVectorizer
### 2 = TF-IDF 

# Models

### Remember to use your own path to your model!

## Random Forest on CountVectorizer 

In [34]:
clf_loaded = load('/Users/jamespavlicek/Desktop/QMST/4320/Project Final/random_forest_model_countvectorizer.joblib')

predictions = clf_loaded.predict(X1)

overall_accuracy = accuracy_score(df["Return Type"], predictions)

print(f"Overall Accuracy of the model on the entire dataset is: {overall_accuracy:.2f}")

df['Random_Forest_CV'] = predictions

Overall Accuracy of the model on the entire dataset is: 0.51


In [35]:
conf_mat = confusion_matrix(df["Return Type"], predictions)

print(conf_mat)

[[2028 1037]
 [1913 1104]]


## Random Forest on TF-IDF

In [36]:
clf_loaded_2 = load('/Users/jamespavlicek/Desktop/QMST/4320/Project Final/random_forest_model_tf_idf.joblib')

predictions_2 = clf_loaded_2.predict(X2)

overall_accuracy = accuracy_score(df["Return Type"], predictions_2)

print(f"Overall Accuracy of the model on the entire dataset is: {overall_accuracy:.2f}")

df['Random_Forest_TF_IDF'] = predictions

print(predictions_2)

Overall Accuracy of the model on the entire dataset is: 0.51
[0 1 0 ... 1 0 1]


In [37]:
conf_mat_2 = confusion_matrix(df["Return Type"], predictions_2)

print(conf_mat_2)

[[2093  972]
 [2031  986]]


## XGBoost on CountVectorizer 

In [38]:
xgboost_loaded_1 = load('/Users/jamespavlicek/Desktop/QMST/4320/Project Final/XGBoost_model_countvectorizer.joblib')

predictions_3 = xgboost_loaded_1.predict(X1)

overall_accuracy = accuracy_score(df["Return Type"], predictions_3)

print(f"Overall Accuracy of the model on the entire dataset is: {overall_accuracy:.2f}")

df['XGBoost_CV'] = predictions_3

print(predictions_3)

Overall Accuracy of the model on the entire dataset is: 0.52
[0 1 0 ... 1 0 1]


In [39]:
conf_mat_3 = confusion_matrix(df["Return Type"], predictions_3)

print(conf_mat_3)

[[2554  511]
 [2437  580]]


## XGBoost on TF-IDF

In [40]:
xgboost_loaded_2 = load('/Users/jamespavlicek/Desktop/QMST/4320/Project Final/XGBoost_model_tf_idf.joblib')

predictions_4 = xgboost_loaded_2.predict(X2)

overall_accuracy = accuracy_score(df["Return Type"], predictions_4)

print(f"Overall Accuracy of the model on the entire dataset is: {overall_accuracy:.2f}")

df['XGBoost_CV_TF_IDF'] = predictions_4

print(predictions_4)

Overall Accuracy of the model on the entire dataset is: 0.51
[0 1 1 ... 0 0 1]


In [41]:
conf_mat_4 = confusion_matrix(df["Return Type"], predictions_4)

print(conf_mat_4)

[[1923 1142]
 [1849 1168]]


## Naive Bayes on CountVectorizer

In [42]:
naive_bayes_loaded_1 = load('/Users/jamespavlicek/Desktop/QMST/4320/Project Final/naive_bayes_model_countvectorizer.joblib')

predictions_5 = naive_bayes_loaded_1.predict(X1)

overall_accuracy = accuracy_score(df["Return Type"], predictions_5)

print(f"Overall Accuracy of the model on the entire dataset is: {overall_accuracy:.2f}")

df['Naive_Bayes_CV'] = predictions_5

print(predictions_5)

Overall Accuracy of the model on the entire dataset is: 0.49
[0 0 1 ... 0 1 0]


In [43]:
conf_mat_5 = confusion_matrix(df["Return Type"], predictions_5)

print(conf_mat_5)

[[2271  794]
 [2286  731]]


## Naive Bayes on TF-IDF

In [44]:
naive_bayes_loaded_2 = load('/Users/jamespavlicek/Desktop/QMST/4320/Project Final/naive_bayes_model_tf_idf.joblib')

predictions_6 = naive_bayes_loaded_2.predict(X2)

overall_accuracy = accuracy_score(df["Return Type"], predictions_6)

print(f"Overall Accuracy of the model on the entire dataset is: {overall_accuracy:.2f}")

df['Naive_Bayes_TF_IDF'] = predictions_6

print(predictions_6)

Overall Accuracy of the model on the entire dataset is: 0.50
[0 0 1 ... 1 1 1]


In [45]:
conf_mat_6 = confusion_matrix(df["Return Type"], predictions_6)

print(conf_mat_6)

[[ 620 2445]
 [ 611 2406]]


## Entire Text Sentiment (Textblob)

In [49]:
from textblob import TextBlob

for index, row in df.iterrows():
    text = row['Largest Text']
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    df.at[index, 'TextBlob_Sentiment_Score'] = sentiment

    if sentiment > 0.03: 
        df.at[index, 'TextBlob_Sentiment_Type'] = int(1)
    else:
        df.at[index, 'TextBlob_Sentiment_Type'] = int(0)

print(df[['Return Type', 'TextBlob_Sentiment_Score', 'TextBlob_Sentiment_Type']])

      Return Type  TextBlob_Sentiment_Score  TextBlob_Sentiment_Type
0               0                  0.063592                      1.0
1               1                  0.057926                      1.0
2               1                  0.073174                      1.0
3               1                  0.053829                      1.0
4               1                  0.042621                      1.0
...           ...                       ...                      ...
6077            1                  0.021044                      0.0
6078            0                  0.025346                      0.0
6079            0                  0.020568                      0.0
6080            1                  0.040332                      1.0
6081            1                  0.033845                      1.0

[6082 rows x 3 columns]


In [50]:
conf_mat_7 = confusion_matrix(df["Return Type"], df["TextBlob_Sentiment_Type"])

print(conf_mat_7)

[[ 837 2228]
 [ 882 2135]]


## Loughran-McDonald Word Count

This model is a frequency analysis of the text compared to a positive / negative word list from Bill McDonald at Notre Dame University. This dataset was derived from 10ks from 1993-2023. Soruce: https://sraf.nd.edu/loughranmcdonald-master-dictionary/

In [48]:
df.shape

(6082, 29)

In [51]:
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer

data = pd.read_csv("/Users/jamespavlicek/Desktop/QMST/4320/Project Final/Loughran-McDonald_MasterDictionary.csv")

testing_df = data.copy()

negative_word_list = []
positive_word_list = []

# Go through each row in the csv and see if there is a postive value in the Negative or Positive row
for index, row in testing_df.iterrows():
    if row['Negative'] > 0:
        negative_word_list.append(row['Word'])
    if row['Positive'] > 0:
        positive_word_list.append(row['Word'])

negative_word_list = [word.lower() for word in negative_word_list]
positive_word_list = [word.lower() for word in positive_word_list]

# Read the text and count the frequency of positive and negative words
def count_neg_and_pos_words(text, negative_word_list, positive_word_list):
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)

    count_of_negative_words = 0 
    count_of_positive_words = 0 
    total_words = len(words)
    for word in words:
        word_lower = word.lower() 
        if word_lower in negative_word_list:
            count_of_negative_words += 1
        elif word_lower in positive_word_list:
            count_of_positive_words += 1
    
    return count_of_positive_words, count_of_negative_words, total_words

for index, row in df.iterrows():
    text = row['Largest Text'] 
    results = count_neg_and_pos_words(text, negative_word_list, positive_word_list)

    # Load the results into the dataframe
    df.at[index, 'Positive_Word_Count_Loughran_McDonald'] = results[0]
    df.at[index, 'Negative_Word_Count_Loughran_McDonald'] = results[1]
    df.at[index, 'Total Word Count_Loughran_McDonald'] = results[2]

    # Calculate the Pos minus Neg Score 
    Pos_neg_score = results[0] - results[1]
    df.at[index, "Word_Pos_minus_Neg_Score_Loughran_McDonald"] = Pos_neg_score 

    pos_word_ratio = results[0] / results[2]
    neg_word_ratio = results[1] / results[2]
    strength_ratio = pos_word_ratio - neg_word_ratio

    if strength_ratio > 0:
        df.at[index, 'Word_Sentiment_Type_Loughran_McDonald'] = 1
    else:
        df.at[index, 'Word_Sentiment_Type_Loughran_McDonald'] = 0

df.head()

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Largest Text,Ticker,Percent Change (1 Week),Return Type,Random_Forest_CV,...,Word_Sentiment_Type_Loughran_McDonald,TextBlob_Sentiment_Score,TextBlob_Sentiment_Type,FinBERT_Positive_Sentence_Count,FinBERT_Neutral_Sentence_Count,FinBERT_Negative_Sentence_Count,FinBERT_Pos_minus_Neg_Score,FinBERT_Sentiment_Type,Chat_GPT_Sentiment,Chat_GPT_Sentiment_binary
0,20211214_10-K_edgar_data_827876_0001663577-21-...,"cleanspark, inc.",20211214,827876,services-computer integrated systems design [7...,item management discussion analysis financial ...,CLSK,-0.106524,0,1,...,0.0,0.063592,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20220225_10-K_edgar_data_1046025_0001046025-22...,heritage financial corp /wa/,20220225,1046025,"savings institutions, not federally chartered ...",item managements discussion analysis financial...,HFWA,0.008755,1,0,...,0.0,0.057926,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210202_10-K_edgar_data_822416_0000822416-21-...,pultegroup inc/mi/,20210202,822416,operative builders [1531],item management discussion analysis financial ...,PHM,0.071569,1,1,...,0.0,0.073174,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20190222_10-K_edgar_data_1099219_0001099219-19...,metlife inc,20190222,1099219,life insurance [6311],item management discussion analysis financial ...,MET,0.010058,1,1,...,0.0,0.053829,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20230214_10-K_edgar_data_1109357_0001109357-23...,exelon corp,20230214,1109357,electric & other services combined [4931],item managements discussion analysis financial...,EXC,0.024268,1,1,...,0.0,0.042621,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
conf_mat_8 = confusion_matrix(df["Return Type"], df["Word_Sentiment_Type_Loughran_McDonald"])

print(conf_mat_8)

[[2895  170]
 [2871  146]]


## Setence Sentiment (FinBERT) (Skipped - Takes 1 minute per row)

source: https://huggingface.co/yiyanghkust/finbert-tone 

In [ ]:
import pandas as pd
import nltk
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# Function to perform sentiment analysis on a row
def analyze_sentiment(index):
    row = df_2.loc[index]
    large_text_string = row['Largest Text']
    sentences = nltk.tokenize.sent_tokenize(large_text_string)
    filtered_sentences = [sentence for sentence in sentences if len(nltk.word_tokenize(sentence)) <= 100]
    
    if filtered_sentences:
        results_3 = nlp(filtered_sentences)
        count = {"Positive": 0, "Neutral": 0, "Negative": 0}
        for item in results_3:
            label = item['label']
            if label in count:
                count[label] += 1      
        return index, count
    else:
        return index, {"Positive": 0, "Neutral": 0, "Negative": 0}

# Load the FinBERT sentiment analysis model
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

# Call the analyze_sentiment() function and put the results to the dataframe
for index in df_2.index:
    index, count = analyze_sentiment(index)
    print(index)
    print(count)
    df.loc[index, 'FinBERT_Positive_Sentence_Count'] = count['Positive']
    df.loc[index, 'FinBERT_Neutral_Sentence_Count'] = count['Neutral']
    df.loc[index, 'FinBERT_Negative_Sentence_Count'] = count['Negative']
    Pos_neg_score = count['Positive'] - count['Negative']
    df.loc[index, "FinBERT_Pos_minus_Neg_Score"] = Pos_neg_score 

    if Pos_neg_score > 0:
        df.loc[index,'FinBERT_Sentiment_Type'] = 1
    else:
        df.loc[index,'FinBERT_Sentiment_Type'] = 0

In [ ]:
conf_mat_9 = confusion_matrix(df["Return Type"], df['FinBERT_Sentiment_Type'])

print(conf_mat_9)

## Chat GPT API (Skipped - 1 minute per row)

Used code from chatgptmax package in order to send text chunks larger than 16k token. For this model you need acsess to Chat GPT API and a API key. Source: https://github.com/victoriadrake/chatgptmax in order to 

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import openai
import os 
import re
import tiktoken
import numpy as np
import time

os.environ['OPENAI_API_KEY'] = 'ENTER_YOUR_API_KEY'

client = OpenAI(api_key='ENTER_YOUR_API_KEY')

load_dotenv()

df['Chat_GPT_Sentiment'] = np.nan
df['Chat_GPT_Sentiment_binary'] = np.nan

#openai.api_key = 'OPENAI_API_KEY'  # Replace with your actual API key

def send(
    prompt=None,
    text_data=None,
    chat_model="gpt-3.5-turbo-0125",
    model_token_limit=16000,
    max_tokens=10000,
):
    """
    Send the prompt at the start of the conversation and then send chunks of text_data to ChatGPT via the OpenAI API.
    If the text_data is too long, it splits it into chunks and sends each chunk separately.

    Args:
    - prompt (str, optional): The prompt to guide the model's response.
    - text_data (str, optional): Additional text data to be included.
    - max_tokens (int, optional): Maximum tokens for each API call. Default is 2500.

    Returns:
    - list or str: A list of model's responses for each chunk or an error message.
    """

    # Check if the necessary arguments are provided
    if not prompt:
        return "Error: Prompt is missing. Please provide a prompt."
    if not text_data:
        return "Error: Text data is missing. Please provide some text data."

    # Initialize the tokenizer
    tokenizer = tiktoken.encoding_for_model(chat_model)

    # Encode the text_data into token integers
    token_integers = tokenizer.encode(text_data)

    # Split the token integers into chunks based on max_tokens
    chunk_size = max_tokens - len(tokenizer.encode(prompt))
    chunks = [
        token_integers[i : i + chunk_size]
        for i in range(0, len(token_integers), chunk_size)
    ]

    # Decode token chunks back to strings
    chunks = [tokenizer.decode(chunk) for chunk in chunks]

    responses = []
    messages = [
        {"role": "user", "content": prompt},
        {
            "role": "user",
            "content": "To provide the context for the above prompt, I will send you text in parts. When I am finished, I will tell you 'ALL PARTS SENT'. Do not answer until you have received all the parts.",
        },
    ]

    for chunk in chunks:
        messages.append({"role": "user", "content": chunk})

        # Check if total tokens exceed the model's limit and remove oldest chunks if necessary
        while (
            sum(len(tokenizer.encode(msg["content"])) for msg in messages)
            > model_token_limit
        ):
            messages.pop(1)  # Remove the oldest chunk

        response = client.chat.completions.create(model=chat_model, messages=messages)
        chatgpt_response = response.choices[0].message
        responses.append(chatgpt_response)

    # Add the final "ALL PARTS SENT" message
    messages.append({"role": "user", "content": "ALL PARTS SENT"})
    response = client.chat.completions.create(model=chat_model, messages=messages)
    final_response = response.choices[0].message
    responses.append(final_response)

    return responses

def text(text):
    """
    Cleans the provided text by removing URLs, email addresses, non-letter characters, and extra whitespace.

    Args:
    - text (str): The input text to be cleaned.

    Returns:
    - str: The cleaned text.
    """
    # Remove URLs
    text = re.sub(r"http\S+", "", text)

    # Remove email addresses
    text = re.sub(r"\S+@\S+", "", text)

    # Remove everything that's not a letter (a-z, A-Z)
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    return text


def summarize_text(tenk_text):
    file_content = text(tenk_text)
    # Define your prompt
    prompt_text = "You are a financial analyst and I am given you a portion of the item 7 of a SEC 10k report. Please summarize this text in around 5 sentences or 100 words. This text will later be used for sentiment analysis so make sure not to remove it in the summary. I will keep sending parts of the document and keep summarizing each part until I stop sending."
    
    # Send the file content to ChatGPT
    responses = send(prompt=prompt_text, text_data=file_content)
    
    # Print the responses
    text_summary = []
    for response in responses:
        #print(f'This is the response: {response}')

        extracted_content = response.content
        #print(f'This is the extracted_content: {extracted_content}')

        text_summary.append(extracted_content)
        
    #print(f'This is the text_summary: {text_summary}.')
    
    return text_summary


def analyze_sentiment(text):
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a financial analyst that conducts sentiment analysis on SEC 10-k documents. You respond in either 'Positive' or 'Negative'."},
        {"role": "user", "content": f"I have a list of summaries from the item 7 of a SEC 10k report. The summaries provided are in the same order as the original 10k's item 7. Do not take into account any financial document I gave you previously, only look at the string of text that will be provided later in this prompt. What is the sentiment of the following text? (keep your answer to one word, Positive or Negative.) {text}"}
    ]
    )

    message_text = completion.choices[0].message.content
    #print(message_text)
    return message_text

#Save the results to the dataframe
for index, row in df_2.iterrows():
    time.sleep(20)
    tenk_text = row["Largest Text"]
    text_summary = summarize_text(tenk_text)
    message_text = analyze_sentiment(text_summary)
    print(f'The {index} 10k is : {message_text}')
    
    df.at[index, 'Chat_GPT_Sentiment'] = message_text

    if message_text == "Positive":
        df.at[index, 'Chat_GPT_Sentiment_binary'] = 1
    else:
        df.at[index, 'Chat_GPT_Sentiment_binary'] = 0

## Export to CSV

In [ ]:
df.head(20)

In [ ]:
df_2.shape

In [ ]:
df = df.drop(columns=['Largest Text'])

df.to_csv('QMST_4320_Project_Final_Results_final.csv', index=False)

## Model Analysis

Trimmed down the dataset and fixed textblob and word sentiment for a more favorable Sensitivity & Specificity

In [53]:
data = pd.read_csv("/Users/jamespavlicek/Desktop/QMST/4320/Project Final/QMST_4320_Project_Final_Results_10_models - QMST_4320_Project_Final_Results.csv")

In [54]:
df_meta = data.copy()

In [55]:
df_meta.head()

,Filename,Company Name,Filed As Of Date,Central Index Key,Standard Industrial Classification,Ticker,Percent Change (1 Week),Return Type_actual,Random_Forest_CV_pred,Random_Forest_TF_IDF_pred,XGBoost_CV_pred,XGBoost_CV_TF_IDF_pred,Naive_Bayes_CV_pred,Naive_Bayes_TF_IDF_pred,TextBlob_Sentiment_Type_adjusted_pred,Word_Sentiment_Type_Loughran_McDonald_adjusted_pred,FinBERT_Sentiment_Type_adjusted_pred,Chat_GPT_pred
0,20211214_10-K_edgar_data_827876_0001663577-21-...,"cleanspark, inc.",20211214,827876,services-computer integrated systems design [7...,CLSK,-0.106524,0,0,0,1,1,1,1,1,0,0,0
1,20220225_10-K_edgar_data_1046025_0001046025-22...,heritage financial corp /wa/,20220225,1046025,"savings institutions, not federally chartered ...",HFWA,0.008755,1,0,0,1,0,0,1,1,0,1,1
2,20210202_10-K_edgar_data_822416_0000822416-21-...,pultegroup inc/mi/,20210202,822416,operative builders [1531],PHM,0.071569,1,1,1,1,1,0,0,1,1,1,1
3,20190222_10-K_edgar_data_1099219_0001099219-19...,metlife inc,20190222,1099219,life insurance [6311],MET,0.010058,1,1,1,1,1,0,0,1,1,0,0
4,20230214_10-K_edgar_data_1109357_0001109357-23...,exelon corp,20230214,1109357,electric & other services combined [4931],EXC,0.024268,1,0,0,1,1,0,1,1,0,0,0


In [56]:
df_meta.shape

(2593, 18)

In [57]:
print(df_meta.columns)

Index(['Filename', 'Company Name', 'Filed As Of Date', 'Central Index Key',
       'Standard Industrial Classification', 'Ticker',
       'Percent Change (1 Week)', 'Return Type_actual',
       'Random_Forest_CV_pred', 'Random_Forest_TF_IDF_pred', 'XGBoost_CV_pred',
       'XGBoost_CV_TF_IDF_pred', 'Naive_Bayes_CV_pred',
       'Naive_Bayes_TF_IDF_pred', 'TextBlob_Sentiment_Type_adjusted_pred',
       'Word_Sentiment_Type_Loughran_McDonald_adjusted_pred',
       'FinBERT_Sentiment_Type_adjusted_pred', 'Chat_GPT_pred'],
      dtype='object')


In [58]:
df = df_meta

In [59]:
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [60]:
# Actual column name
actual_column = 'Return Type_actual'
actual = df[actual_column].astype(bool)

# List your predicted columns here
predicted_columns = ['Random_Forest_CV_pred', 'Random_Forest_TF_IDF_pred', 'XGBoost_CV_pred',
       'XGBoost_CV_TF_IDF_pred', 'Naive_Bayes_CV_pred',
       'Naive_Bayes_TF_IDF_pred', 'TextBlob_Sentiment_Type_adjusted_pred',
       'Word_Sentiment_Type_Loughran_McDonald_adjusted_pred',
       'FinBERT_Sentiment_Type_adjusted_pred', 'Chat_GPT_pred']


## Each model Preformance on Test Set

In [61]:
results = [] 

for predicted_column in predicted_columns:
    predicted = df[predicted_column].astype(bool)
    
    # No Information Rate (NIR)
    NIR = actual.value_counts(normalize=True).max()
    
    # Accuracy
    accuracy = accuracy_score(actual, predicted)
    
    # Sensitivity (Recall)
    sensitivity = recall_score(actual, predicted)
    
    # Specificity
    tn, fp, fn, tp = confusion_matrix(actual, predicted).ravel()
    specificity = tn / (tn + fp)
    
    # Append the results to the list
    results.append({'Predicted Column': predicted_column, 
                    'NIR': NIR, 
                    'Accuracy': accuracy, 
                    'Sensitivity': sensitivity, 
                    'Specificity': specificity})

results_df = pd.DataFrame(results, columns=['Predicted Column', 'NIR', 'Accuracy', 'Sensitivity', 'Specificity'])

results_df

,Predicted Column,NIR,Accuracy,Sensitivity,Specificity
0,Random_Forest_CV_pred,0.512919,0.490551,0.561362,0.423308
1,Random_Forest_TF_IDF_pred,0.512919,0.490551,0.561362,0.423308
2,XGBoost_CV_pred,0.512919,0.476668,0.786223,0.182707
3,XGBoost_CV_TF_IDF_pred,0.512919,0.509063,0.755344,0.275188
4,Naive_Bayes_CV_pred,0.512919,0.499422,0.270784,0.716541
5,Naive_Bayes_TF_IDF_pred,0.512919,0.495565,0.294537,0.686466
6,TextBlob_Sentiment_Type_adjusted_pred,0.512919,0.502892,0.520190,0.486466
7,Word_Sentiment_Type_Loughran_McDonald_adjusted...,0.512919,0.501735,0.535234,0.469925
8,FinBERT_Sentiment_Type_adjusted_pred,0.512919,0.489009,0.440222,0.535338
9,Chat_GPT_pred,0.512919,0.493251,0.520190,0.467669
